This markdown is an explaination of how the optimization model works for selecting Draftking lineups. To run this file, make sure you have python installed. In addition, you will need the python library pandas(used for manipulating in excel data) and pulp (this actually does the optimization, https://github.com/coin-or/pulp).

Quickest way to have these installed is to have pip installed. Then 'pip install pandas' and 'pip install pulp'

In [2]:
from pulp import *
import pandas as pd
import re , csv
from draftkings import *
import time

In [10]:
print "Starting optimization model"
print "----------------------------------"
print "Reading Data"
player_data=pd.read_csv("../Input/DKSalaries.csv")
prob = pulp.LpProblem('NFL', pulp.LpMaximize)



Starting optimization model
----------------------------------
Reading Data


In Pulp, everything is built as a LP model. Hence from here, all we did was say we are having a maximization LP file. The player_data is reading in the data from DKSalaries. This input will be changed on a week to week basis.  

Starting optimization model
----------------------------------
Reading Data


In [13]:
players={}
total_budget=50000
positions=["QB", "RB", "WR", "TE", "DST"]

salaries={}
position_constraints={}

for position in positions:
	salaries[position]=''
	position_constraints[position]=''

 
num_players=''

teams=player_data.teamAbbrev.unique()
team_constraints={}


freq_limit=10
objective_function=''
total_cost=''
l_collection=LineupCollection()

Setting Up variables for future use


In [14]:
for team in teams:
	team_constraints[team]={}
	team_constraints[team]['QB']=''
	team_constraints[team]['WR']=''

In [15]:
print team_constraints

{'MIN': {'QB': '', 'WR': ''}, 'MIA': {'QB': '', 'WR': ''}, 'CAR': {'QB': '', 'WR': ''}, 'ATL': {'QB': '', 'WR': ''}, 'DET': {'QB': '', 'WR': ''}, 'NYJ': {'QB': '', 'WR': ''}, 'DEN': {'QB': '', 'WR': ''}, 'BAL': {'QB': '', 'WR': ''}, 'NYG': {'QB': '', 'WR': ''}, 'TEN': {'QB': '', 'WR': ''}, 'NO': {'QB': '', 'WR': ''}, 'DAL': {'QB': '', 'WR': ''}, 'NE': {'QB': '', 'WR': ''}, 'BUF': {'QB': '', 'WR': ''}, 'SEA': {'QB': '', 'WR': ''}, 'CHI': {'QB': '', 'WR': ''}, 'PIT': {'QB': '', 'WR': ''}, 'JAX': {'QB': '', 'WR': ''}, 'OAK': {'QB': '', 'WR': ''}, 'GB': {'QB': '', 'WR': ''}, 'WAS': {'QB': '', 'WR': ''}, 'KC': {'QB': '', 'WR': ''}, 'PHI': {'QB': '', 'WR': ''}, 'TB': {'QB': '', 'WR': ''}, 'CLE': {'QB': '', 'WR': ''}, 'LAC': {'QB': '', 'WR': ''}, 'IND': {'QB': '', 'WR': ''}, 'ARI': {'QB': '', 'WR': ''}, 'SF': {'QB': '', 'WR': ''}, 'LAR': {'QB': '', 'WR': ''}}


So far all we have down is defined a bunch of variables. We have done nothing to our actual LP model. Notice in the team constraints, we are getting ready to build the Stacking constraints. In this type of constraint, we want to make sure QB matches his WR. Hence, we need to group the QBs and WR's per team. As you can see, we have built in expressions for each team in our for loop and initialized them to empty. As we loop through our player list, say Tom Brady, we will find his respective team, identify position in that team, then set the constraint such that

team_constraints[team]['QB']<=sum(team_constraints[team]['WR'])

In [16]:
for rownum, row in player_data.iterrows():
	variable=str('x'+str(rownum))
	variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer')

	player=Player(row, str(variable))
	players[str(variable)]=player
	#print player.name

	num_players += variable

	player_points = player.projected*variable
	total_cost+= player.salary*variable
	objective_function += player_points
	 
	position_constraints[player.position]+=variable
	#Team Stacking Organization
	if player.position=='QB' and player.projected!=0:
		team_constraints[player.team]['QB']+=variable

	if player.position=='WR':
		team_constraints[player.team]['WR']+=variable

Here we loop through the player data . Each row represents some information about the player. We represent a player a a binary variable , such as x45, x33, x12, etc. Here we also see how our objective function is being built. We are continously incrementing the player_points which is points*binary variable, where if the variable is 1 if we select that variable. 


The total cost is our budget constraint


The Stacking constraint is preparing us to state that QB and WR are on the same team. Note that there are 2QBs per team in the DK csv. We say, player.projected!=0 to ensure we only care about the primary QB getting most of the minutes

In [21]:
prob += lpSum(objective_function)
prob += (total_cost<=50000)
prob += (num_players==9)

In [22]:
print prob

NFL:
MAXIMIZE
14.1*x0 + 7.7*x1 + 22.0*x10 + 16.3*x100 + 14.6*x101 + 1.8*x103 + 10.4*x104 + 9.8*x105 + 12.0*x106 + 12.1*x107 + 12.52*x109 + 15.64*x11 + 6.7*x111 + 2.04*x112 + 1.1*x116 + 11.48*x120 + 0.1*x121 + 5.22*x123 + 10.7*x124 + 14.6*x125 + 16.2*x127 + 4.0*x128 + 10.3*x129 + 9.5*x13 + 11.7*x130 + 20.6*x131 + 0.9*x134 + 3.32*x135 + 20.94*x14 + 6.2*x142 + 11.6*x146 + 6.6*x147 + 3.0*x148 + 20.54*x15 + -0.22*x150 + 0.3*x151 + 9.9*x152 + 11.7*x155 + 7.1*x157 + 8.0*x158 + 12.9*x159 + 11.9*x16 + 0.6*x160 + 25.3*x161 + 17.6*x163 + 5.9*x164 + 7.1*x165 + 2.3*x166 + 18.68*x17 + 6.2*x173 + 1.3*x175 + 5.5*x176 + 6.0*x177 + 9.5*x179 + 14.3*x18 + 9.8*x181 + 2.6*x184 + 1.0*x186 + 18.9*x19 + 4.2*x190 + 3.8*x191 + 7.0*x195 + 1.8*x196 + 6.6*x197 + 14.1*x198 + 22.9*x199 + 32.2*x2 + 5.3*x20 + 1.8*x201 + 4.0*x202 + 8.1*x203 + 4.0*x204 + 7.2*x205 + 9.5*x207 + 3.3*x208 + 49.6*x21 + 6.4*x210 + 12.9*x213 + 3.0*x214 + 7.6*x215 + 1.1*x216 + 7.3*x217 + 12.4*x218 + 15.3*x219 + 20.6*x22 + 11.5*x220 + 1.9*x222 + 

So far, we are saying maximize projected points under a budget, with 9 players

In [24]:
print "Building Constraints"
min_limits=[1, 2, 3, 1, 1]
#Actual stacking constraints
for team in team_constraints:
    #print team_constraints[team]['QB'], team 
    prob += (team_constraints[team]['QB'] <= team_constraints[team]['WR'] )
#Position constraints
for i, position in enumerate(positions):
    if position =="QB" or position=="DST":
        prob+= (position_constraints[position]<=1)
        prob+= (position_constraints[position]>=1)
    else:
        prob+= (position_constraints[position]>=min_limits[i])
        prob+= (position_constraints[position]<=min_limits[i]+1)

Building Constraints


Here we build the stacking constraint, and the position constraints to make sure we have a feasible lineup.
Notice we can only have 1 QB and 1DST. Due to the Flex position, this forces us to have the "else" portion where we say min_limits[i]+1. Since the number of players must be 9, this ensures the flex must be a WR/TE/RB

In [30]:
lineups=[]
num_lineups=10
print "Writing Lineup"
for i in range(1,num_lineups+1):
	print 'Iteration %d'% i
	fileLP="NFL_X%d.lp"%i			
	#prob.writeLP(fileLP)
	optimization_result = prob.solve()
	#assert optimization_result == pulp.LpStatusOptimal

	lineup=[]
	selected_vars=[]

	diversity_constraint=''
	div_limit=3
	lineup_values=[]
	for var in prob.variables():
		if 'x' not in str(var):
			continue
		if var.varValue:
			player.count+=1
			frequency_constraint=''
			frequency_constraint+=player.count*var+var
			prob+=(frequency_constraint<=freq_limit)

			selected_vars.append(var)
			player=players[str(var)]
			#print player.name
			lineup.append(player)

			var.varValue=0
		#Force diversity s.t no than two lineups can share more than 3 players
	diversity_constraint=sum([var for var in selected_vars])				
	prob+=(diversity_constraint<=div_limit)
	#print len(lineup)
	lineups.append(lineup)



Writing Lineup
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10


We have our constraints ready, now it's time to optimize. prob.solve() solves our LP model. for var in prob.variables(): goes through each binary variable(all the players in our case). the if var.varValue: means this value is true, which means this player is selected

We need to keep track of two things, a diversity limit and frequency limit. To ensure lineup diversity, we want to say any two lineups are say 3 players different. As we go through more and more iterations, the diversity constraint is added every time per iteration to force more diversity. 


frequency_constraint+=player.count*var+var Increments the player count to ensure we cap the number of times we use a player

The rest of the file just writes out the output to a csv